# Ratings_cleaning musi zostać uruchomiony jako pierwszy
Notebook `Ratings_cleaning.ipynb` usuwa niektóre z wystawionych ocen, co zmniejsza ich liczbę u użytkowników. Następnie usuwa użytkowników, którzy mają mniej niż 20 ocen i zapisuje nowy zbiór danych do pliku `user_features_2.csv`.

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('../data/user_features_train_2.csv')
df_test = pd.read_csv('../data/user_features_test_2.csv')

In [3]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198928 entries, 0 to 198927
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 198928 non-null  int64  
 1   num_rating             198928 non-null  int64  
 2   avg_rating             198928 non-null  float64
 3   weekend_watcher        198928 non-null  int64  
 4   type_of_viewer         198928 non-null  object 
 5   genre_Action           197761 non-null  float64
 6   genre_Adventure        198024 non-null  float64
 7   genre_Animation        172725 non-null  float64
 8   genre_Comedy           198517 non-null  float64
 9   genre_Crime            196860 non-null  float64
 10  genre_Documentary      73077 non-null   float64
 11  genre_Drama            198866 non-null  float64
 12  genre_Family           187087 non-null  float64
 13  genre_Fantasy          194577 non-null  float64
 14  genre_History          172639 non-nu

# Uzupełnienie brakujących ocen 

In [5]:
genre_columns = ['genre_Action','genre_Adventure','genre_Animation','genre_Comedy','genre_Crime','genre_Documentary','genre_Drama','genre_Family','genre_Fantasy','genre_History','genre_Horror','genre_Musical','genre_Mystery','genre_Romance','genre_Science Fiction','genre_TV Movie','genre_Thriller','genre_War','genre_Western']

for col in genre_columns:
    df_train[col] = df_train[col].fillna(df_train['avg_rating'])
    df_test[col] = df_test[col].fillna(df_test['avg_rating'])

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198928 entries, 0 to 198927
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 198928 non-null  int64  
 1   num_rating             198928 non-null  int64  
 2   avg_rating             198928 non-null  float64
 3   weekend_watcher        198928 non-null  int64  
 4   type_of_viewer         198928 non-null  object 
 5   genre_Action           198928 non-null  float64
 6   genre_Adventure        198928 non-null  float64
 7   genre_Animation        198928 non-null  float64
 8   genre_Comedy           198928 non-null  float64
 9   genre_Crime            198928 non-null  float64
 10  genre_Documentary      198928 non-null  float64
 11  genre_Drama            198928 non-null  float64
 12  genre_Family           198928 non-null  float64
 13  genre_Fantasy          198928 non-null  float64
 14  genre_History          198928 non-nu

# Enkodowanie kolumny 'type_of_viewer'

In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

def encoder(df):
    encoded = ohe.fit_transform(df[['type_of_viewer']])
    encoded = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(['type_of_viewer']))
    encoded.index = df.index
    df = df.drop(columns=['type_of_viewer'])
    df = pd.concat([df, encoded], axis=1)

    return df

df_train = encoder(df_train)
df_test = encoder(df_test)

print(df_train.head(3))
print(df_test.head(3))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198928 entries, 0 to 198927
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   198928 non-null  int64  
 1   num_rating               198928 non-null  int64  
 2   avg_rating               198928 non-null  float64
 3   weekend_watcher          198928 non-null  int64  
 4   genre_Action             198928 non-null  float64
 5   genre_Adventure          198928 non-null  float64
 6   genre_Animation          198928 non-null  float64
 7   genre_Comedy             198928 non-null  float64
 8   genre_Crime              198928 non-null  float64
 9   genre_Documentary        198928 non-null  float64
 10  genre_Drama              198928 non-null  float64
 11  genre_Family             198928 non-null  float64
 12  genre_Fantasy            198928 non-null  float64
 13  genre_History            198928 non-null  float64
 14  genr

# Dołączenie ostatnich 20 ocenionych filmów ze zbioru ratings_clean_zlib.h5

In [6]:
ratings_df = pd.read_parquet('../data/ratings_clean_groupped_20.parquet')
df_train = pd.merge(df_train, ratings_df, on='userId')
df_test = pd.merge(df_test, ratings_df, on='userId')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198832 entries, 0 to 198831
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   198832 non-null  int64  
 1   num_rating               198832 non-null  int64  
 2   avg_rating               198832 non-null  float64
 3   weekend_watcher          198832 non-null  int64  
 4   genre_Action             198832 non-null  float64
 5   genre_Adventure          198832 non-null  float64
 6   genre_Animation          198832 non-null  float64
 7   genre_Comedy             198832 non-null  float64
 8   genre_Crime              198832 non-null  float64
 9   genre_Documentary        198832 non-null  float64
 10  genre_Drama              198832 non-null  float64
 11  genre_Family             198832 non-null  float64
 12  genre_Fantasy            198832 non-null  float64
 13  genre_History            198832 non-null  float64
 14  genr

# Normalizacja danych numerycznych

In [7]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = ['num_rating', 'avg_rating']
columns_to_normalize.extend(genre_columns)
print('Columns to normalize:', columns_to_normalize)

scaler = StandardScaler()

df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
print(df_train.head(3))

Columns to normalize: ['num_rating', 'avg_rating', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Music', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western']
   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       1   -0.068675   -0.347979                0      0.926736   
1       2   -0.383417    1.210645                0      0.713096   
2       3   -0.047456   -0.228499                0      0.045472   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0        -0.375240        -0.179705     -0.402570     0.892727   
1         0.940526         1.581734      0.973277     0.410751   
2         0.066743         0.383741     -0.669363    -0.692235   

   genre_Documentary  ...  genre_TV Movie  genre_Thriller  genre_War  \
0          -

## Dla danych testowych

In [ ]:
df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])
print(df_test.head(3))

# Konwersja kolumny weekend_watcher do float64

In [8]:
df_train['weekend_watcher'] = df_train['weekend_watcher'].astype('float64')
df_test['weekend_watcher'] = df_test['weekend_watcher'].astype('float64')

In [9]:
print('\', \''.join(df_train.columns.to_list()))

userId', 'num_rating', 'avg_rating', 'weekend_watcher', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Music', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western', 'type_of_viewer_negative', 'type_of_viewer_neutral', 'type_of_viewer_positive', 'movies_seq', 'ratings_seq', 'ts_seq


# Zapis do pliku

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198832 entries, 0 to 198831
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   198832 non-null  int64  
 1   num_rating               198832 non-null  float64
 2   avg_rating               198832 non-null  float64
 3   weekend_watcher          198832 non-null  float64
 4   genre_Action             198832 non-null  float64
 5   genre_Adventure          198832 non-null  float64
 6   genre_Animation          198832 non-null  float64
 7   genre_Comedy             198832 non-null  float64
 8   genre_Crime              198832 non-null  float64
 9   genre_Documentary        198832 non-null  float64
 10  genre_Drama              198832 non-null  float64
 11  genre_Family             198832 non-null  float64
 12  genre_Fantasy            198832 non-null  float64
 13  genre_History            198832 non-null  float64
 14  genr

In [11]:
df_train.to_parquet('../datasets/user_features_clean_train.parquet', compression='brotli')

In [ ]:
df_test.info()

In [ ]:
df_test.to_parquet('../datasets/user_features_clean_train.parquet', compression='brotli')